In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/Users/melaniecostello/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/melaniecostello/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.read_csv('../train_dev_data/train_data.csv')
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,g936neasyy,2013-05-12,20130512210934,2013-05-13,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Mac Desktop,Chrome,other
1,duq2vabpp2,2013-03-02,20130302054534,NaN,FEMALE,31.0,facebook,0,en,direct,direct,untracked,Web,iPad,Mobile Safari,NDF
2,xiymwcsklc,2011-05-17,20110517211429,NaN,-unknown-,105.0,facebook,2,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF
3,8kkcksa0dw,2013-12-02,20131202180650,2013-12-11,-unknown-,37.0,basic,0,en,sem-brand,google,omg,Web,iPad,Mobile Safari,US
4,zk8qx61d9m,2013-11-07,20131107183734,NaN,FEMALE,25.0,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF


In [3]:
dev = pd.read_csv('../train_dev_data/dev_data.csv')
dev.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,vwcvba5tbi,2013-05-10,20130510181208,2013-05-17,MALE,43.0,facebook,0,en,direct,direct,linked,Web,Mac Desktop,Chrome,US
1,lh49nmvbhr,2014-05-30,20140530230300,2014-12-15,FEMALE,35.0,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome,CA
2,peowrqmisu,2013-03-15,20130315150346,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Mac Desktop,Safari,NDF
3,568wanlyy1,2014-02-10,20140210064831,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,NDF
4,ihi4gy0dyl,2014-01-01,20140101193508,2014-01-06,FEMALE,29.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,other


In [4]:
# Split off training labels & dev labels
train_labels = train.country_destination
dev_labels = dev.country_destination

In [5]:
# Explore relationships between variables and outcome
#pd.crosstab(train.language, train.country_destination)

In [6]:
# pd.crosstab(train.gender, train.country_destination)

In [7]:
# pd.crosstab(train.first_device_type, train.country_destination)

In [8]:
# Transform variables we want to use into binary
test_cols = ['language', 'gender']
tr_binary_vars = pd.get_dummies(train[['language', 'gender']], columns=test_cols)

dev_binary_vars = pd.get_dummies(dev[['language', 'gender']], columns=test_cols)
dev_binary_vars.head()
# tr_binary_vars.head()

,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hu,language_id,...,language_pt,language_ru,language_sv,language_th,language_tr,language_zh,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
# Note that training data has two extra languages in it - ca & hr
# We need to add dummy columns for ca & hr to the dev data

# List of columns in transformed training data & dev data
tr_list = list(tr_binary_vars.columns.values)
dev_list = list(dev_binary_vars.columns.values)

# difference between the sets
np.setdiff1d(tr_list, dev_list)

array(['language_ca', 'language_hr'], 
      dtype='|S16')

In [10]:
dev_binary_vars.insert(0, 'language_ca', 0, allow_duplicates=False)
dev_binary_vars.insert(9, 'language_hr', 0, allow_duplicates=False)
dev_binary_vars.head()

,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hr,...,language_pt,language_ru,language_sv,language_th,language_tr,language_zh,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
# Create arrays based on the variables we want to use
train_array = np.array(tr_binary_vars)
train_label_array = np.array(train_labels)

dev_array = np.array(dev_binary_vars)
dev_label_array = np.array(dev_labels)

In [12]:
# Try a model
def mod_test(k_vals, train_data, train_labels, dev_data, dev_labels=None):

### STUDENT START ###

    # We want to evaluate a variety of values for k, so we need
    # to enclose our work in a loop.
    for elem in k_vals:
        
        # Create a classifier object, fit our training data &
        # initialize a variable to hold the predictions
        knn = KNeighborsClassifier(n_neighbors=elem)
        knn.fit(train_data, train_labels)
        preds = knn.predict(dev_data)
        
        # We evaluate the accuracy for each value of k by comparing
        # the predictions and the labels, then updating values for
        # correct and total
        correct, total = 0, 0
        for pred, label in zip(preds, dev_labels):
            if pred == label: 
                correct += 1
            total += 1
        print 'For k=%s, total: %3d  correct: %3d  accuracy: %3.2f' %(elem, total, correct, 1.0*correct/total)

k_vals = [7, 10, 15, 20, 50]
mod_test(k_vals, train_array[:5000], train_label_array[:5000], dev_array[:2000], dev_label_array[:2000])

For k=7, total: 2000  correct: 772  accuracy: 0.39
For k=10, total: 2000  correct: 1158  accuracy: 0.58
For k=15, total: 2000  correct: 1159  accuracy: 0.58
For k=20, total: 2000  correct: 1159  accuracy: 0.58
For k=50, total: 2000  correct: 1159  accuracy: 0.58


In [13]:
# Try another model
def mod_test2(train_data, train_labels, dev_data, dev_labels):
    
    # LOGISTIC REGRESSION
    log = LogisticRegression()
    
    C_options = {'C': np.arange(0.1, 1, 0.1)}
    log_grid = GridSearchCV(log, C_options)

    log_grid.fit(train_data, train_labels)
    log_preds = log_grid.predict(dev_data)
    
    # Output best param
    print "Best value for C: %.2f" %log_grid.best_params_['C']
    print "F1 score for Logistic Regression: %.3f" %metrics.f1_score(dev_labels, log_preds, average="weighted") + "\n"
    

mod_test2(train_array[:2000], train_label_array[:2000], dev_array[:2000], dev_label_array[:2000])

/Users/melaniecostello/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


Best value for C: 0.10
F1 score for Logistic Regression: 0.425



/Users/melaniecostello/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Baseline submission

In [14]:
# Use KNN to predict on test data
# Get test data
test = pd.read_csv('../unzipped_files/test_users.csv')
test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [15]:
# Process test data so it matches layout of training data
test_cols = ['language', 'gender']
test_binary_vars = pd.get_dummies(test[['language', 'gender']], columns=test_cols)
test_binary_vars.head()

,language_-unknown-,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,...,language_pt,language_ru,language_sv,language_th,language_tr,language_zh,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [16]:
tr_binary_vars.head()

,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hr,...,language_pt,language_ru,language_sv,language_th,language_tr,language_zh,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
# List of columns in transformed training data & dev data
tr_list = list(tr_binary_vars.columns.values)
test_list = list(test_binary_vars.columns.values)

# difference between the sets
# Train list has language_hr, language_is
# Test list has language_-unknown-
np.setdiff1d(test_list, tr_list)

# Drop columns that don't exist in both
tr_binary_vars = tr_binary_vars.drop(["language_hr", "language_is"], axis=1)
test_binary_vars = test_binary_vars.drop("language_-unknown-", axis=1)

In [18]:
# Create arrays based on the variables we want to use
train_array = np.array(tr_binary_vars)
train_label_array = np.array(train_labels)

test_array = np.array(test_binary_vars)

In [71]:
test_array

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=uint8)

In [28]:
def mod_score(k_val, train_data, train_labels, test_data):
        
    # Create a classifier object, fit our training data &
    # initialize a variable to hold the predictions
    knn = KNeighborsClassifier(n_neighbors=k_val)
    knn.fit(train_data, train_labels)
    preds = knn.predict(test_data)
        
    return preds

test_preds = mod_score(15, train_array[:5000], train_label_array[:5000], test_array)

In [36]:
t = pd.DataFrame(test_preds, columns=['country'])
ids = test[['id']]
output = pd.concat([ids, t], axis='columns')
output.to_csv('Spring_W207_Group.csv', index=False)
